In [1]:
import pandas as pd

In [2]:
metadata = pd.read_json('metadata.json')

In [4]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 4 columns):
ac_code       679 non-null int64
ac_name       679 non-null object
state_code    679 non-null object
state_name    679 non-null object
dtypes: int64(1), object(3)
memory usage: 21.3+ KB


In [6]:
metadata.sample(3)

,ac_code,ac_name,state_code,state_name
415,34,DHOD,S20,Rajasthan
247,178,PANDHANA,S12,Madhya Pradesh
599,61,Jubilee Hills,S29,Telangana


In [8]:
df_1 = pd.read_json('assembly-elections-2018.json', lines=True)

In [10]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9150 entries, 0 to 9149
Data columns (total 5 columns):
ac       9150 non-null int64
name     9150 non-null object
party    9150 non-null object
state    9150 non-null object
votes    9150 non-null int64
dtypes: int64(2), object(3)
memory usage: 357.5+ KB


In [23]:
df_1.sample(3)

,ac,name,party,state,votes
458,64,KAMLESH KUMAR NAGARCHI,Independent,S26,553
9048,105,VIKRANTH SAMIDLA,Independent,S29,185
8391,95,KATTA SRINIVAS,Yekikrutha Sankshema Rashtriya Praja Party,S29,520


In [27]:
"df_1.shape={} metadata.shape={}".format(df_1.shape, metadata.shape)

'df_1.shape=(9150, 5) metadata.shape=(679, 4)'

In [36]:
# add state_name and ac_name columns
df_2 = df_1.merge(metadata.rename(columns={'ac_code':'ac','state_code':'state'}))

In [37]:
df_2.shape

(9150, 7)

In [78]:
df_2.sample(3)

,ac,name,party,state,votes,ac_name,state_name
7588,79,VEERA PRASAD,Bahujan Samaj Party,S29,1406,Gadwal,Telangana
6677,19,RAJENDRA SINGH,Bharat Vahini Party,S20,129,Sadulpur,Rajasthan
5164,143,FOJARAM,Bahujan Mukti Party,S20,656,Bhinmal,Rajasthan


In [80]:
# number of candidates in each state
df_2.state_name.value_counts()

Madhya Pradesh    3129
Rajasthan         2473
Telangana         1940
Chhattisgarh      1359
Mizoram            249
Name: state_name, dtype: int64

In [91]:
df = df_2 # dataframe we are working on

# get winner from each ac
tmp = df.groupby(['state_name','ac_name'], as_index=False).apply(lambda g: g.nlargest(1,'votes'))

# number of constituencies and candidates
pd.DataFrame(dict(constituencies=tmp.state_name.value_counts(), candidates=df_2.state_name.value_counts()))

,candidates,constituencies
Madhya Pradesh,3129,230
Rajasthan,2473,198
Telangana,1940,119
Chhattisgarh,1359,90
Mizoram,249,40


In [225]:
def by_state(df):
    byac = df.groupby('ac')
    byparty = df.groupby('party')
    tmp = (byparty[['votes']].sum()/df.votes.sum()).round(3).rename(columns={'votes':'vote_pct'})
    tmp = tmp.assign(
            won          =byac.apply(lambda g : g.nlargest(1,'votes')).party.value_counts(),
            candidates   =byparty.count().ac,
            runnerup     =byac.apply(lambda g : g.nlargest(2,'votes').iloc[1]).party.value_counts(),
            not_contested=(df.ac.unique().shape[0] - byparty.count().ac).map(lambda a: 0 if a < 0 else a)
        ).dropna().astype({'won':int,'runnerup':int}).sort_values(by='vote_pct', ascending=False)
    return tmp

df_2.groupby('state_name').apply(by_state)


vote_pct  candidates  \
state_name     party                                                       
Chhattisgarh   Indian National Congress                0.430          90   
               Bharatiya Janata Party                  0.330          90   
               Janta Congress Chhattisgarh (J)         0.076          57   
               Bahujan Samaj Party                     0.039          35   
Madhya Pradesh Bharatiya Janata Party                  0.410         230   
               Indian National Congress                0.409         229   
               Independent                             0.058        1094   
               Bahujan Samaj Party                     0.050         227   
               Samajwadi Party                         0.013          52   
Mizoram        Mizo National Front                     0.376          40   
               Indian National Congress                0.302          40   
               Independent                             0.229          63   
               Bharatiya Janata Party                  0.080          39   
Rajasthan      Indian National Congress                0.393         194   
               Bharatiya Janata Party                  0.388         199   
               Independent                             0.095         830   
               Bahujan Samaj Party                     0.040         189   
               Rashtriya Loktantrik Party              0.024          58   
               Communist Party of India  (Marxist)     0.012          28   
               Bhartiya Tribal Party                   0.007          11   
               Rashtriya Lok Dal                       0.003           2   
Telangana      Telangana Rashtra Samithi               0.469         119   
               Indian National Congress                0.284          99   
               Bharatiya Janata Party                  0.070         118   
               Telugu Desam                            0.035          13   
               Independent                             0.033         675   
               All India Forward Bloc                  0.008          19   

                                                    not_contested  runnerup  \
state_name     party                                                          
Chhattisgarh   Indian National Congress                         0        13   
               Bharatiya Janata Party                           0        68   
               Janta Congress Chhattisgarh (J)                 33         2   
               Bahujan Samaj Party                             55         4   
Madhya Pradesh Bharatiya Janata Party                           0       113   
               Indian National Congress                         1        98   
               Independent                                      0         5   
               Bahujan Samaj Party                              3         6   
               Samajwadi Party                                178         5   
Mizoram        Mizo National Front                              0        12   
               Indian National Congress                         0        18   
               Independent                                      0         9   
               Bharatiya Janata Party                           1         1   
Rajasthan      Indian National Congress                         5        72   
               Bharatiya Janata Party                           0       105   
               Independent                                      0        11   
               Bahujan Samaj Party                             10         2   
               Rashtriya Loktantrik Party                     141         2   
               Communist Party of India  (Marxist)            171         2   
               Bhartiya Tribal Party                          188         1   
               Rashtriya Lok Dal                              197         1   
Telangana      Telangana Rasht